In [ ]:
#!pip uninstall 'qiskit-aqua' -y
!pip install --upgrade 'qiskit[visualization,optimization]'

In [ ]:
#!pip install --upgrade 'qiskit-aqua[skquant]'

In [ ]:
!pip install scikit-quant

In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

import numpy as np
import math
from docplex.mp.model import Model
from math import log

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [2]:
# Quantum Aggregator Wrapper

# Import algorithms 
import grovers_search as grovers
import optim_wrapper as optimization


def aggregator(algorithm, dict_details):
    if algorithm == 'grovers':
        result = grovers.grovers_search(dict_details)
    if algorithm == 'optimizer':
        result = optimization.optimize_portfolio(dict_details)
    return result


# ETF Optimization (Use_case_v3) 


## Binary representation of integers limited to budget amount

(Based on H2 Hypothesis in paper v1 and stock grouping in paper v2)

$\begin{aligned}
\min_{x \in \mathbb{N}^n}  q x^T \Sigma x - \mu^T x\\
\end{aligned}$

subject to:

$\begin{aligned}
\ \sum_{i=0}^{n} S^T x \le B,\quad x \in \mathbb{N}^n,\quad S \in \mathbb{R}^n,\quad B \in \mathbb{R} \qquad \approx_{k\to\infty}  \qquad \sum_{i=0}^{n}  ceil( \frac{k}{\overline{S}}s_i) x_i \le floor(\frac{k}{\overline{S}}B) ,\quad k \in \mathbb{R},\quad ceil(.), floor(.) \in \mathbb{N}\\
\end{aligned}$

integer variables $x_i$ bounded to:

$\begin{aligned}
0 \le x_i \le 2^{floor(log_{2}(\frac{B}{\min_{j}{s_j}}))}-1
\end{aligned}$

where we use the following notation:

- $x \in \mathbb{N}^n$ denotes the vector of integer decision variables, which indicate which assets to pick and how much ($x[i] > 0$) and which not to pick ($x[i] = 0$),
- $y$ represents the binary decomposition of $x$
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets,
- $\Sigma \in \mathbb{R}^{n \times n}$ specifies the covariances between the assets,
- $q > 0$ controls the risk appetite of the decision maker,
- $S$ defined as the prices related to the (grouped) stocks ( $\overline{S}$ is the mean of the prices),
- $B$ denotes the budget, i.e. the maximum amount of money to be spent purchasing assets.
- $ceil$ is the operator that rounds up to the nearest integer
- $floor$ is the operator that rounds down to the nearest integer
- $k$ is a scaling factor used to tune the precision of the conversions to integer


In [3]:
# prepare problem instance
n = 5            # number of assets
q = 0.5          # risk factor
budget = 2250  # budget
# automatic penalty
#penalty = budget**n    # scaling of penalty term. TODO: find optimum
#print('Penalty:', penalty)

In [4]:

# example instance
s_real = np.array([60.04, 55.87, 139.72, 32.46, 111.07])
grouping = np.array([2, 3, 1, 4, 1])
s=grouping*s_real

print(' ̃s:', s)

 ̃s: [120.08 167.61 139.72 129.84 111.07]


In [5]:
print(s.mean())

133.664


In [6]:
budget_bits = int(np.log2(budget/np.min(s))) #TODO: int rounds down. I guess this is just excluding high volatility solutions.
print("Budget variables per asset:", budget_bits)

Budget variables per asset: 4


In [7]:
mu = np.array([0.00290, 0.00206, 0.00033, 0.00263, -0.00012])
sigma = np.array([
    [ 0.00016, 0.00014, 0.00013, 0.00016, 0.00015],
    [ 0.00014, 0.00016, 0.00013, 0.00016, 0.00014],
    [ 0.00013, 0.00013, 0.00028, 0.00013, 0.00028],
    [ 0.00016, 0.00016, 0.00013, 0.00021, 0.00015],
    [ 0.00015, 0.00014, 0.00028, 0.00015, 0.00030]
])

In [8]:
# COBYLA K=7 full
# optimal function value: 0.0168
# optimal value: [12.  1.  1.  0.  2.]
initial_point = np.array([
  2.59053218,  3.07383748, -0.02830739, -3.63309921, -2.59853671, -3.20797840,
  3.33314935, -1.35133976,  3.63283071,  3.69698830, -1.23352425,  3.31862756,
  4.19931877,  1.43270108, -3.98425304, -1.57150233, -0.65458941, -0.81643120,
  4.94757565,  2.07617259,  0.07179877,  0.07303629,  2.91093525,  0.12795972,
  0.08646192, -2.99756703,  1.50578149, -1.04225831, -0.18934945,  2.62426510,
 -3.28927912,  2.79095167, -3.68730170,  2.82009121,  0.07565113, -0.15255559,
  0.03094703,  1.77756513, -1.74105261, -0.64174123, -2.06943340, -2.34060495,
 -1.07889976, -0.29862926, -0.52227792,  0.57689027, -0.29581687, -1.24656532,
  1.28653548,  2.82191423,  1.74868550, -2.30037444,  2.41060747, -0.33028446
  ])

In [9]:
# IMFIL K=7 circular
# optimal function value: 0.01958
# optimal value: [5. 6. 0. 4. 0.]
initial_point = np.array([
 -0.10443451,  0.15794821, -3.14159265,  3.14159265,  3.14159265,  0.08134372,
 -2.05583004,  1.96418066,  3.14159265,  2.62961285, -1.53778771,  0.15115402,
 -2.34264387,  1.09693660, -0.50155170, -1.14538203,  1.72507405,  0.47064835,
 -3.14159265, -1.89449310, -2.59013929,  0.04510715,  3.14159265, -3.14159265,
 -0.56119690,  3.13799737, -3.14159265, -3.14159265,  0.87437460, -0.33974137,
 -0.03436667,  3.14159265,  2.22961905,  3.14159265,  3.14159265,  3.14159265,
  1.80531028,  1.17137820,  3.13190004,  2.44889741,  1.21174119,  0.45292839,
 -1.03871678, -2.36174585,  3.14159265,  2.24771505, -0.74333817,  3.14159265,
  3.14159265, -3.14159265,  0.00435256, -0.24178762,  3.14159265,  0.17713373
  ])

In [10]:
# create docplex model
mdl = Model('portfolio_optimization')

x = mdl.integer_var_list((f'x{i}' for i in range(n)), lb=0, ub=2**budget_bits-1)

objective = mdl.sum([mu[i]*x[i] for i in range(n)])
objective -= q * mdl.sum([sigma[i,j]*x[i]*x[j] for i in range(n) for j in range(n)])
mdl.maximize(objective)

k = 7
norm = s.mean()/k
mdl.add_constraint(mdl.sum(x[i] * math.ceil(s[i]/norm) for i in range(n)) <= math.floor(budget/norm))

# import the classical optimizer
from qiskit.algorithms.optimizers import COBYLA, SLSQP, BOBYQA, IMFIL, SNOBFIT

#maxiter=2 # TODO: just for testing purposes
optim_dict = {
  "docplex_mod": mdl,
  "optimizer":SLSQP
}

In [11]:
# Call the aggregator with 'optimizer' as the algorithm of choice
results = aggregator('optimizer', optim_dict)

### Parameters:
{'quantum_instance': 'qasm_simulator', 'shots': 1024, 'print': True, 'solver': 'vqe', 'entanglement': 'circular', 'depth': 1, 'maxiter': 1000, 'alpha': 0.35, 'docplex_mod': <docplex.mp.model.Model object at 0x7fe66190df10>, 'optimizer': <class 'qiskit.algorithms.optimizers.slsqp.SLSQP'>}
### Original problem:
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: portfolio_optimization

Maximize
 obj: 0.002900000000 x0 + 0.002060000000 x1 + 0.000330000000 x2
      + 0.002630000000 x3 - 0.000120000000 x4 + [ - 0.000160000000 x0^2
      - 0.000280000000 x0*x1 - 0.000260000000 x0*x2 - 0.000320000000 x0*x3
      - 0.000300000000 x0*x4 - 0.000160000000 x1^2 - 0.000260000000 x1*x2
      - 0.000320000000 x1*x3 - 0.000280000000 x1*x4 - 0.000280000000 x2^2
      - 0.000260000000 x2*x3 - 0.000560000000 x2*x4 - 0.000210000000 x3^2
      - 0.000300000000 x3*x4 - 0.000300000000 x4^2 ]/2
Subject To
 c0: 7 x0 + 9 x1 + 8 x2 + 7 x3 + 6 x4 <= 117

Bounds
       x0

MemoryError: Unable to allocate 256. PiB for an array with shape (134217728, 134217728) and data type complex128

In [ ]:
results['result'].variables_dict

In [ ]:
print()
print('Integer results (amount of groups of stocks): x')
x_val = [results['result'].variables_dict[f'x{i}'] for i in range(5)]
print(x_val)

print()
print('Amount of individual stock:')
x_ind = grouping*x_val
print(x_ind)

print()
print('Amount invest in each stock:')
invest = s_real*x_ind
print(invest)

print()
print('Total invest:')
print(invest.sum())

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table

In [ ]:
qiskit.__qiskit_version__

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table

In [ ]:
qiskit.__qiskit_version__